In [1]:
import pandas as pd
import numpy as np

In [2]:
df2 = pd.read_csv('/content/drive/MyDrive/Tower_Sentiment.csv')
#Dropping columns that do not need to be read
df2.drop(df2.columns[[4,5,6,7,8]], axis=1, inplace=True)
df2.head()

<ipython-input-2-d0ac50016251>:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/drive/MyDrive/Tower_Sentiment.csv')


,Conversation ID,Source,Text,Timestamp,FullSentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,user,I,2023-04-17T21:15:34.492,Neutral
1,00036309-c615-45b0-bd95-d4669b72e66b,bot,Hmm.. sorry I couldn't find an answer for that!,2023-04-17T21:16:08.778,NaN
2,00036309-c615-45b0-bd95-d4669b72e66b,bot,"To have a guided conversation instead, please ...",2023-04-17T21:16:09.801,NaN
3,00036309-c615-45b0-bd95-d4669b72e66b,bot,Are you an existing Tower customer?,2023-04-17T21:16:10.82,NaN
4,00036309-c615-45b0-bd95-d4669b72e66b,user,Yes,2023-04-17T21:16:28.619,NaN


In [3]:
df2['Text'] = df2['Text'].fillna('')
grouped_df = df2.groupby('Conversation ID').agg({'Text': '. '.join, 'FullSentiment': 'first'}).reset_index()
grouped_df.head()
#len(grouped_df)

,Conversation ID,Text,FullSentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,I. Hmm.. sorry I couldn't find an answer for t...,Neutral
1,000715a4-1d49-4a54-8ee8-140808202926,I have a pending claim and the company fixing ...,Neutral
2,000c670b-4513-49d4-bf3a-daa371d4ef52,Hi there. Kia Ora! 👋. I can help you navigate ...,Positive
3,000d5541-0607-463a-b05c-aeb888a69233,Hi I actually wanted to cancel my policy. I ca...,Positive
4,0016ac21-a8ae-476d-aa75-8c4b62ce175a,"Hi Charlie bot, im trying to add my airpoints ...",Positive


In [4]:
df2['Text'] = df2['Text'].fillna('')

# Filter out rows where the 'Source' column is 'bot'
df2 = df2.drop(df2[df2['Source'] == 'bot'].index)

# Group by 'ID' and concatenate the 'Text' for each group separated by a period ('.')
grouped_df = df2.groupby('Conversation ID').agg({'Text': '. '.join, 'FullSentiment': 'first'}).reset_index()

# Print the first few rows of the grouped DataFrame
print(grouped_df.head())

                        Conversation ID  \
0  00036309-c615-45b0-bd95-d4669b72e66b   
1  000715a4-1d49-4a54-8ee8-140808202926   
2  000c670b-4513-49d4-bf3a-daa371d4ef52   
3  000d5541-0607-463a-b05c-aeb888a69233   
4  0016ac21-a8ae-476d-aa75-8c4b62ce175a   

                                                Text FullSentiment  
0                                             I. Yes       Neutral  
1  I have a pending claim and the company fixing ...       Neutral  
2  Hi there. I hv a question do ur cover for trav...      Positive  
3           Hi I actually wanted to cancel my policy      Positive  
4  Hi Charlie bot, im trying to add my airpoints ...      Positive  


In [5]:
text_paragraph = grouped_df['Text'][5]
print(text_paragraph)

My insurance payment was scheduled for yesterday but it didn't come out due to the public holiday, when will this payment come out??. Would this payment usually be impacted by a public holiday? Am I still insured if it hasn't come out yet?. Terri McMeikan. 3/2/1992. terrimcmeikan92@gmail.com. Thank you. P00004356530. Ok. no. Hi Maryanne!. Thanks for coming to help. Okay cool, will it effect my cover being late?. Awesome, thank you I was quite concerned for a moment. There is enough in there for the payment to come out Tuesday
 Have a lovely afternoon


In [6]:
df_filled = grouped_df.fillna(" ")
old_word = 'Neural'
new_word = 'Neutral'
# Find rows containing the target word and replace it with the new word
df_filled['FullSentiment'] = df_filled['FullSentiment'].replace(old_word, new_word)

old_word = 'Positve'
new_word = 'Positive'
# Find rows containing the target word and replace it with the new word
df_filled['FullSentiment'] = df_filled['FullSentiment'].replace(old_word, new_word)

old_word = 'positive'
new_word = 'Positive'
# Find rows containing the target word and replace it with the new word
df_filled['FullSentiment'] = df_filled['FullSentiment'].replace(old_word, new_word)

# Save the modified DataFrame back to the CSV file
df_filled.to_csv('/content/drive/MyDrive/Tower_Sentiment_New.csv', index=False)

#Removing all mistakes from the dataset where names were not spelt right
specific_word = 'Neural'
df_filled = df_filled[~df_filled['FullSentiment'].str.contains(specific_word, case=False)]
specific_word = 'Positve'
df_filled = df_filled[~df_filled['FullSentiment'].str.contains(specific_word, case=False)]
print(len(df_filled))
df_filled.head()

9731


,Conversation ID,Text,FullSentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,I. Yes,Neutral
1,000715a4-1d49-4a54-8ee8-140808202926,I have a pending claim and the company fixing ...,Neutral
2,000c670b-4513-49d4-bf3a-daa371d4ef52,Hi there. I hv a question do ur cover for trav...,Positive
3,000d5541-0607-463a-b05c-aeb888a69233,Hi I actually wanted to cancel my policy,Positive
4,0016ac21-a8ae-476d-aa75-8c4b62ce175a,"Hi Charlie bot, im trying to add my airpoints ...",Positive


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
#train and test data split
train_data, test_data = train_test_split(df_filled, test_size=0.3, random_state=42)

In [8]:
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed

# Fit and transform the vectorizer on the training data
X_train = vectorizer.fit_transform(train_data['Text'])
y_train = train_data['FullSentiment']

# Transform the vectorizer on the test data
X_test = vectorizer.transform(test_data['Text'])
y_test = test_data['FullSentiment']

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#LinearRegression
linearR = LinearRegression()
#Create a Naive Bayes classifier
classifier = MultinomialNB()
#LogisticRegression
logistic = LogisticRegression()
#Decision Tree
decision = DecisionTreeClassifier()
#Random forest
forest = RandomForestClassifier()
#SVM
svm = SVC()

# Train the classifier on the training data
#linearR.fit(X_train, y_train)
classifier.fit(X_train, y_train)
logistic.fit(X_train, y_train)
decision.fit(X_train, y_train)
forest.fit(X_train, y_train)
svm.fit(X_train, y_train)

SVC()

In [10]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test data
y_pred = classifier.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes:", accuracy)

# l_pred = linearR.predict(X_test)
# # Calculate accuracy
# accuracy = accuracy_score(y_test, l_pred)
# print("Linear Regression:", accuracy)

logistic_pred = logistic.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, logistic_pred)
print("Logistic Regression:", accuracy)

d_pred = decision.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, d_pred)
print("Decision Tree:", accuracy)

f_pred = forest.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, f_pred)
print("Random Forest:", accuracy)

svm_pred = svm.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, svm_pred)
print("Svm:", accuracy)

# Generate a classification report
print("Naive Bayes Report:")
print(classification_report(y_test, y_pred))

print("Logistic Regression Report:")
print(classification_report(y_test, logistic_pred))

print("Decision Tree Report:")
print(classification_report(y_test, d_pred))

print("Random Forest Report:")
print(classification_report(y_test, f_pred))

print("SVM Report:")
print(classification_report(y_test, svm_pred))


Naive Bayes: 0.9787671232876712
Logistic Regression: 0.9787671232876712
Decision Tree: 0.9667808219178082
Random Forest: 0.9787671232876712
Svm: 0.9787671232876712
Naive Bayes Report:
              precision    recall  f1-score   support

                   0.98      1.00      0.99      2858
    Negative       0.00      0.00      0.00        14
     Neutral       0.00      0.00      0.00        25
    Positive       0.00      0.00      0.00        23

    accuracy                           0.98      2920
   macro avg       0.24      0.25      0.25      2920
weighted avg       0.96      0.98      0.97      2920

Logistic Regression Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

                   0.98      1.00      0.99      2858
    Negative       0.00      0.00      0.00        14
     Neutral       0.00      0.00      0.00        25
    Positive       0.00      0.00      0.00        23

    accuracy                           0.98      2920
   macro avg       0.24      0.25      0.25      2920
weighted avg       0.96      0.98      0.97      2920

Decision Tree Report:
              precision    recall  f1-score   support

                   0.98      0.99      0.98      2858
    Negative       0.00      0.00      0.00        14
     Neutral       0.00      0.00      0.00        25
    Positive       0.00      0.00      0.00        23

    accuracy                           0.97      2920
   macro avg       0.24      0.25      0.25      2920
weighted avg       0.96      0.97      0.96      2920

Random Forest Report:
              precision    recall  f1-score   support

                   0.98      1.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [11]:
new_text = "no"
new_text_features = vectorizer.transform([new_text])

#Bayes
predicted_sentiment = classifier.predict(new_text_features)[0]
print("Predicted Sentiment Bayes:", predicted_sentiment)
#logistic
predicted_sentiment = logistic.predict(new_text_features)[0]
print("Predicted Sentiment logistic:", predicted_sentiment)
#Decision treee
predicted_sentiment = decision.predict(new_text_features)[0]
print("Predicted Sentiment Decision tree:", predicted_sentiment)
#Forest
predicted_sentiment = forest.predict(new_text_features)[0]
print("Predicted Sentiment forest:", predicted_sentiment)
#Svm
predicted_sentiment = svm.predict(new_text_features)[0]
print("Predicted Sentiment Svm:", predicted_sentiment)

Predicted Sentiment Bayes:  
Predicted Sentiment logistic:  
Predicted Sentiment Decision tree:  
Predicted Sentiment forest:  
Predicted Sentiment Svm:  
